In [ ]:
!pip install kmodes

In [ ]:
import random
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
from itertools import cycle, islice
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from kmodes.kprototypes import KPrototypes
import missingno as msno 
from sklearn.decomposition import PCA
import time
from sklearn import metrics
from itertools import permutations  
from sklearn.metrics import confusion_matrix
from sklearn.utils.linear_assignment_ import linear_assignment
import csv 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import os
import urllib.request
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/adult1.csv', 'adult.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/sat.csv', 'satellite.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/GermanData1.csv', 'german.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/heart.csv', 'heart.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/kddcup991.csv', 'kddcup.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/shuttle.csv', 'shuttle.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/phenotypic.csv', 'soybean.csv')

['.config',
 '.ipynb_checkpoints',
 'shuttle.csv',
 'adult.csv',
 'german.csv',
 'satellite.csv',
 'kddcup.csv',
 'soybean.csv',
 'heart.csv',
 'sample_data']

In [ ]:
def accuracy(true_row_labels, predicted_row_labels):

    cm = confusion_matrix(true_row_labels, predicted_row_labels)
    indexes = linear_assignment(_make_cost_m(cm))
    total = 0
    for row, column in indexes:
        value = cm[row][column]
        total += value

    return (total * 1. / np.sum(cm))

def _make_cost_m(cm):
    s = np.max(cm)
    return (- cm + s)

In [ ]:
def InclusionProposedApproach(x1,n):
    pca = PCA(n_components=1)
    x = pca.fit_transform(x1)
    mi=0
    for i in range(0,len(x)):
        if x[i][0]<mi:
            mi=x[i][0]
    for i in range(0,len(x)):
        x[i][0]-=mi
        x[i][0]+=1
    N=len(x)
    f=[0]*len(x)
    m=0
    m1=-1
    pi=[0]*N
    while m1!=m:
        m1=m
        xx=0
        for i in range(N):
            xx+=x[i][0]*(1-f[i])
        c=n-m
        for i in range(N):
            if f[i]==0:
                pi[i]=c*x[i][0]/xx
                if pi[i]>=1:
                    f[i]=1
            else:
                pi[i]=1
        m=sum(f)
    return pi

def InclusionShastri(x1,n):
    x=np.array(x1)
    x=np.transpose(x)
    for i in range(0,len(x)):
        x[i] = [float(j)/sum(x[i]) for j in x[i]]
    ma=[]
    for i in range(0,len(x)):
        ma.append(max(x[i]))
    x=np.transpose(x)
    dev=[0]*len(x)
    
    for i in range(0,len(x)):
        for j in range(0,len(x[i])):
            dev[i]+=(ma[j]-x[i][j])
    pi=[]
    su=0
    for i in dev:
        su+=1/i
    for i in dev:
        pi.append(n*(1/i)/su)
        
    return pi

def InclusionNigam(x1,n):
    x=np.array(x1)
    x=np.transpose(x)
    for i in range(0,len(x)):
        x[i] = [float(j)/sum(x[i]) for j in x[i]]
    x=np.transpose(x)
    dev=[0]*len(x)
    for i in range(0,len(x)):
        for j in range(0,len(x[i])):
            dev[i]+=x[i][j]
    pi=[]
    su=0
    for i in dev:
        su+=i
    for i in dev:
        pi.append(n*i/su)
    return pi

In [ ]:
def row_echelon(A):
    r, c = A.shape
    if r == 0 or c == 0:
        return A
    for i in range(len(A)):
        if A[i,0] != 0:
            break
    else:
        B = row_echelon(A[:,1:])
        return np.hstack([A[:,:1], B])

    if i > 0:
        ith_row = A[i].copy()
        A[i] = A[0]
        A[0] = ith_row
    A[0] = A[0] / A[0,0]
    return A
def fastflightcube(p,X):
    nc=len(X[0])
    nr=len(X)
    u=[0]*nc
    uset=[0]*nc
    la1=la2=10**200
    la=eps=10**-9
    v=free=-1
    X=np.array(X)
    X=row_echelon(X)
    for i in range(nr-1,-1,-1):
        lead=0
        for j in range(0,nc):
            if X[i][j]==0:
                lead+=1
            else:
                break
    if lead<nc:
        v=0.0
        for j in range(lead+1,nc):
            if uset[j]==0:
                uset[j]=1
                free*=-1
                u[j]=free
            v-=u[j]*X[i][j]
        u[lead]=v/X[i][lead]
        uset[lead]=1
    for i in range(0,nc):
        if uset[i]==0:
            free*=-1
            u[i]=free
        else:
            break
    for i in range(0,nc):
        if u[i]>0:
            la1=min(la1,(1-p[i])/u[i])
            la2=min(la2,p[i]/u[i])
        if u[i]<0:
            la1=min(la1,-p[i]/u[i])
            la2=min(la2,(p[i]-1)/u[i])
    if la2/(la1+la2)>random.uniform(0, 1):
        la=la1
    else:
        la=-la2
    for i in range(0,nc):
        p[i]+=la*u[i]
        if p[i]<eps:
            p[i]=0
        if p[i]>1-eps:
            p[i]=1
    return p
def flightphase(X,P):
    n=len(P)
    NN=len(X[0])
    index=[]
    p=P.copy()
    for i in range(0,n):
        index.append(i)
    e=10**(-12)
    d=0
    random.shuffle(index)
    for i in range(d,n):
        if p[index[i]]<e or p[index[i]]>1-e:
            index[d],index[i]=index[i],index[d]
            d+=1
    while d<n:
        hm=min(NN+1,n-d)
        if hm<=NN:
            d=n
            break
        if hm>1:
            p_s=[0]*hm
            i_s=[0]*hm
            B=[[0 for i in range(hm)] for j in range(hm-1)]
            for i in range(0,hm):
                i_s[i]=index[d+i]
                for j in range(0,hm-1):
                    B[j][i]=X[i_s[i]][j]/P[i_s[i]]
                p_s[i]=p[i_s[i]]
            p_s=fastflightcube(p_s,B)
            for i in range(0,hm):
                p[i_s[i]]=p_s[i]
            hl=d+hm
            for i in range(d,hl):
                if p[index[i]]<e or p[index[i]]>1-e:
                    index[d],index[i]=index[i],index[d]
                    d+=1
        else:
            if p[index[d]]>random.uniform(0, 1):
                p[index[d]]=1
            else:
                p[index[d]]=0
            d=n
    for i in range(0,n):
        if p[index[i]]>1-e:
            p[index[i]]=1
        if p[index[i]]<e:
            p[index[i]]=0
    return p

def cube(X,P):
    n=len(P)
    NN=len(X[0])
    index=[]
    p=P.copy()
    for i in range(0,n):
        index.append(i)
    e=10**(-12)
    d=0
    random.shuffle(index)
    for i in range(d,n):
        if p[index[i]]<e or p[index[i]]>1-e:
            index[d],index[i]=index[i],index[d]
            d+=1
    while d<n:
        hm=min(NN+1,n-d)
        if hm>1:
            p_s=[0]*hm
            i_s=[0]*hm
            B=[[0 for i in range(hm)] for j in range(hm-1)]
            for i in range(0,hm):
                i_s[i]=index[d+i]
                for j in range(0,hm-1):
                    B[j][i]=X[i_s[i]][j]/P[i_s[i]]
                p_s[i]=p[i_s[i]]
            p_s=fastflightcube(p_s,B)
            for i in range(0,hm):
                p[i_s[i]]=p_s[i]
            hl=d+hm
            for i in range(d,hl):
                if p[index[i]]<e or p[index[i]]>1-e:
                    index[d],index[i]=index[i],index[d]
                    d+=1
        else:
            if p[index[d]]>random.uniform(0, 1):
                p[index[d]]=1
            else:
                p[index[d]]=0
            d=n
    n1=round(sum(p))
    s=[0]*n1
    count=0
    for i in range(0,n):
        if p[index[i]]>1-e:
            s[count]=index[i]+1
            count+=1
    return s

In [ ]:
filename="heart.csv"
SamplingSize=100

In [ ]:
cat=[]
if filename=="heart.csv":
    cat=[1,2,5,6,8,10,11,12] #for heart
if filename=="german.csv":
    cat=[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18,19]#german
if filename=="shuttle.csv" or filename=="satellite.csv" :
    cat=[]
if filename=="kddcup.csv":
    cat=[1,2,3,6,11,20,21]#for kdd
if filename=="adult.csv":
    cat=[1,3,5,6,7,8,9,13]
if filename=="soybean.csv":
    cat=[0,1,3,4,5,6,7,8,9,12,15,16,17,18,19,20]

In [ ]:
fields = [] 
rows = [] 
with open(filename, 'r') as csvfile: 
    csvreader = csv.reader(csvfile) 
    fields = next(csvreader) 
    for row in csvreader: 
        rows.append(row) 
X=[]
Y=[]
Z=[]
Al=[]
for i in range(0,len(rows)):
    xy=[]
    for j in range(0,len(rows[i])-1):
        xy.append(float(rows[i][j]))
    Al.append(xy.copy())


x=SamplingSize
pi=InclusionProposedApproach(Al,x)
a=cube(Al,pi)
X=[]
Y=[]
Z=[]
Al=[]
for i in range(0,len(rows)):
    xy=[]
    if i+1 in a:
        for j in range(0,len(rows[i])-1):
            xy.append(float(rows[i][j]))
        X.append(xy.copy())
    else:
        for j in range(0,len(rows[i])-1):
            xy.append(float(rows[i][j]))
        Z.append(xy.copy())
    Y.append(int(rows[i][-1]))
    Al.append(xy.copy())

numberofcluster=len(set(Y))
K=numberofcluster
if filename=='dataset/label/shuttle.csv':
    for i in range(0,len(Y)):
        Y[i]-=1
if len(cat)!=0:
    kk=KPrototypes(n_clusters=K)
    kk.fit(np.array(X),categorical=cat)
    c1=list(kk.predict(np.array(Al),categorical=cat))
    xxx=accuracy(c1,Y)
    print(xxx)
    
else:
    kk=KMeans(n_clusters=K)
    kk.fit(np.array(X))
    c1=list(kk.predict(np.array(Al)))
    xxx=accuracy(c1,Y)
    print(xxx)

0.5610561056105611
